## Code to stream the Covid-19 related tweets and save to database

### Uncomment and run this to install libraries in jupyter notebook if running for the first time

In [0]:
# !pip3 install tweepy
# !pip3 install textblob
# !pip3 install nltk
# !pip3 instal dataset

##Code to collect tweets begins here:

In [31]:

import tweepy
# sys.setdefaultencoding('utf8')
import time
import json
import dataset
from textblob import TextBlob
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

dbAllTweets = dataset.connect("sqlite:///tweetsfor24Hrs.db")


# Reading twitter API's credentials from a file

fileReader = open('twitter_secret.txt','r')
lines = (fileReader.read())
lines = lines.split()
consumer_key = lines[0]
consumer_secret = lines[1]
access_token = lines[2]
access_token_secret =lines[3]
fileReader.close()

# OAuth process, using the keys and tokens
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Creation of the actual interface, using authentication
api = tweepy.API(auth)

start_time = time.time()
print(start_time)
limit = 86400  # One day in seconds
totalTweets = 0
positive = 0
negative = 0


class StreamListener(tweepy.StreamListener):
    def on_status(self, status):
        #         if status.text.startswith('RT'):     #Filtering out re-tweets
        #             return
        global positive
        global negative
        global totalTweets

        description = status.user.description
        loc = status.user.location
        text = status.text
        coords = status.coordinates
        name = status.user.screen_name
        user_created = status.user.created_at
        followers = status.user.followers_count
        id_str = status.id_str
        created = status.created_at
        retweets = status.retweet_count
        bg_color = status.user.profile_background_color

        if (time.time() - start_time) > limit:
            print(time.time())
            print("Total tweets =", totalTweets)
            return False
        blob = TextBlob(text)
        sent = blob.sentiment

        polarity = sent.polarity
        subjectivity = sent.subjectivity
        nltkpolarity = sid.polarity_scores(text)['compound']

        if polarity > 0:
            positive += 1
        if polarity < 0:
            negative += 1

        totalTweets += 1
        if totalTweets % 1000 == 0:
            print("Total tweets = ", totalTweets)
            print("Positive = ",positive/totalTweets*100, " %")
            print("Negative = ",negative/totalTweets*100, " %")

        if coords is not None:
            coords = json.dumps(coords)

        table = dbAllTweets["tweets"]
        table.insert(dict(
            user_description=description,
            user_location=loc,
            coordinates=coords,
            text=text,
            user_name=name,
            user_created=user_created,
            user_followers=followers,
            id_str=id_str,
            created=created,
            retweet_count=retweets,
            user_bg_color=bg_color,
            polarity = sent.polarity,
            subjectivity = sent.subjectivity,
            nltkPolarity = nltkpolarity


        ))
        # print(text + ' => ' + str(polarity))
        return



    def on_error(self, status_code):
        if status_code == 420:
            return False




stream_listener = StreamListener()
stream = tweepy.Stream(auth=api.auth, listener=stream_listener)
keywordsListA = ["covid", 'Covid',"COVID", 'corona', 'CORONA', 'Corona', 'pandemic',  'Pandemic', 'PANDEMIC' ]

while True:
    try:
        stream.filter(track=keywordsListA, stall_warnings=True)
    except (ProtocolError, AttributeError):
        continue

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
1587844765.455323
RT @JPLuisi1: Wow!

Man-of-the-people @fordnation thinks that those exercising their democratic rights as Canadians are a ‘bunch of yahoos.… 0.125 0.6239
RT @meganoticiascl: 📺 #MeganoticiasActualiza Sábado | Cyberdelincuentes se aprovechan y lanzan estafa para cobrar bono COVID-19 

📡 Señal e… 0.0 0.0
RT @RSSorg: Rashtriya Swayamsevak Sangh COVID-19 seva till 24 April 2020 :
Places served : 55,725
Dedicated Volunteers : 3,00,809
Ration ki… 0.0 0.4588
RT @viet_t_nguyen: To all the people who said that Asian Americans deserve racism because sone of them are racist or that Chinese are racis… 0.0 -0.8442
RT @Michaella_Moore: We leave campus for 5 minutes and the neighborhood gentrifiers have turned our sacred yard in a COVID park . Please ge… 0.0 0.2732
RT @rtenews: A further 52 people with Covid-19 have died in Ireland, 42 of which were lab confi

NameError: ignored